In [2]:
local = True

In [3]:
from keras.models import load_model
import pandas as pd
import numpy as np

Using TensorFlow backend.
/Users/tianhaozhou/Documents/mlenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
def encode_brand(data, brand_dict):
    res = []
    for d in data:
        row = np.zeros((len(brand_dict)))
        if d in brand_dict:
            row[brand_dict[d]] = 1
        res.append(row)
    output = np.array(res)
    return output

In [5]:
def encode_shipping(data):
    res = [[i] for i in data]
    output = np.array(res)
    return output

In [6]:
def encode_condition(data, condition_dict):
    res = []
    for d in data:
        row = np.zeros((len(condition_dict)))
        row[condition_dict[d]] = 1
        res.append(row)
    output = np.array(res)
    return output

In [7]:
def encode_category(data, dicts, level):
    res = []
    for d in data:
        row = []
        for lv in range(level):
            exrow = [0 for i in range(len(dicts[lv]))]
            if type(d) == str:
                clist = d.split('/')
                if lv < len(clist) and clist[lv] in dicts[lv]:
                    exrow[dicts[lv][clist[lv]]] = 1
            row.extend(exrow)
        res.append(row)
    output = np.array(res)
    return output

In [8]:
test_raw_path = 'data/test.tsv' if local else '/data/test.tsv'
test_raw = pd.read_csv(test_raw_path, sep='\t')

In [9]:
train_raw_path = 'data/train.tsv' if local else '/data/train.tsv'
train_raw = pd.read_csv(train_raw_path, sep='\t')

In [10]:
train_brand_list = list(set(train_raw['brand_name']))
test_brand_list = list(set(test_raw['brand_name']))
print('training brand size: ', len(train_brand_list))
print('testing brand size: ', len(test_brand_list))
print('common brand size: ', len(list(set(train_brand_list)&set(test_brand_list))))
total_brand_list = list(set(train_brand_list)|set(test_brand_list))
brand_dict = {}
for b in total_brand_list:
    brand_dict[b] = len(brand_dict)
print('brand dict size: ', len(brand_dict))

training brand size:  4810
testing brand size:  3901
common brand size:  3421
brand dict size:  5290


In [11]:
total_category = []
total_category.extend(train_raw['category_name'].tolist())
total_category.extend(test_raw['category_name'].tolist())
print('total category count: ', len(total_category))
dicts = []
for c in total_category:
    if type(c) == str:
        clist = c.split('/')
        level_of_category = len(clist)
        while level_of_category > len(dicts):
            dicts.append({})
        for i in range(level_of_category):
            current = clist[i]
            if current not in dicts[i]:
                dicts[i][current] = len(dicts[i])
print('level of dicts: ', len(dicts))

total category count:  2175894
level of dicts:  5


In [12]:
total_item_condition = []
total_item_condition.extend(train_raw['item_condition_id'])
total_item_condition.extend(test_raw['item_condition_id'])
print('total item condition id count: ', len(total_item_condition))
print(list(set(total_item_condition)))
condition_list = list(set(total_item_condition))
condition_dict = {}
for i in range(len(condition_list)):
    condition_dict[condition_list[i]] = i
print('condition dict: ', condition_dict)

total item condition id count:  2175894
[1, 2, 3, 4, 5]
condition dict:  {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}


In [32]:
model = load_model('complete_model.h5')

In [35]:
size = len(test_raw)
trunk = 100
res = []
for i in range((size//trunk) + 1):
    if i % 100 == 0:
        print(i, '/', size//100)
    start = i*trunk
    end = min((i+1)*trunk, size)
    test_condition = encode_condition(test_raw['item_condition_id'][start:end], condition_dict)
    test_shipping = encode_shipping(test_raw['shipping'][start:end])
    test_condition = np.hstack([test_condition, test_shipping])
    test_category = encode_category(test_raw['category_name'][start:end], dicts, 2)
    test_brand = encode_brand(test_raw['brand_name'][start:end], brand_dict)
    tmp = model.predict(x=[test_brand, test_category, test_condition], verbose=0)
    res.extend(sum(tmp.tolist(), []))
print(len(res))

0 / 6933
100 / 6933
200 / 6933
300 / 6933
400 / 6933
500 / 6933
600 / 6933
700 / 6933
800 / 6933
900 / 6933
1000 / 6933
1100 / 6933
1200 / 6933
1300 / 6933
1400 / 6933
1500 / 6933
1600 / 6933
1700 / 6933
1800 / 6933
1900 / 6933
2000 / 6933
2100 / 6933
2200 / 6933
2300 / 6933
2400 / 6933
2500 / 6933
2600 / 6933
2700 / 6933
2800 / 6933
2900 / 6933
3000 / 6933
3100 / 6933
3200 / 6933
3300 / 6933
3400 / 6933
3500 / 6933
3600 / 6933
3700 / 6933
3800 / 6933
3900 / 6933
4000 / 6933
4100 / 6933
4200 / 6933
4300 / 6933
4400 / 6933
4500 / 6933
4600 / 6933
4700 / 6933
4800 / 6933
4900 / 6933
5000 / 6933
5100 / 6933
5200 / 6933
5300 / 6933
5400 / 6933
5500 / 6933
5600 / 6933
5700 / 6933
5800 / 6933
5900 / 6933
6000 / 6933
6100 / 6933
6200 / 6933
6300 / 6933
6400 / 6933
6500 / 6933
6600 / 6933
6700 / 6933
6800 / 6933
6900 / 6933
693359


In [40]:
output_dict = {'test_id': test_raw['test_id'], 'price': res}
output_dataframe = pd.DataFrame(output_dict)
output_dataframe.to_csv('submission.csv')